In [1]:
# Add path candidates to sys.path to import utility / helper modules
import sys
sys.path.append('/workspace')

In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [3]:
"""
Version 1:
- used methods: DP

"""
INF = float('inf')

# Use ~turn to get opposit player
# ~ (bit-wise not operator) is very fast
PLAYER_X = 0
PLAYER_O = ~PLAYER_X

# Preference pattern
WIN = 1
DRAW = 0
LOSE = -1


# Convert given 3*3 game board to an integer in [0, 19682]
#
#   a00 3^8  a01 3^7  a02 3^6
#   a10 3^5  a11 3^4  a12 3^3
#   a20 3^2  a21 3^1  a22 3^0
#
# ret = a00*(3^8) + a01*(3^7) + ... + a21*(3^1) + a22*(3^0)
def flatten(board):
    ret = 0
    for y in range(3):
        for x in range(3):
            cell = board[y][x]
            ret *= 3
            ret += 1 if cell == PLAYER_X else 2 if cell == PLAYER_O else 0

    return ret


# Check whether player of turn made a line
def judge(board, turn):
    # Check rows
    for y in range(3):
        row = board[y]
        if turn == row[0] == row[1] == row[2]:
            return True

    # Check columns
    for x in range(3):
        if turn == board[0][x] == board[1][x] == board[2][x]:
            return True

    # Check diagonals(\, /)
    return (turn == board[0][0] == board[1][1] == board[2][2]  # \
            or turn == board[0][2] == board[1][1] == board[2][0])  # /


# Solution for TICTACTOE
def solution(board):
    # Preprocess
    nx, no = 0, 0
    for y in range(3):
        board[y] = list(board[y])
        for x in range(3):
            if board[y][x] == 'o':
                board[y][x] = PLAYER_O
                no += 1
            elif board[y][x] == 'x':
                board[y][x] = PLAYER_X
                nx += 1
            else:
                board[y][x] = None

    # Player X take first on init
    # nx == no: X
    # nx >  no: O
    # nx <  no: X
    turn = PLAYER_X
    if nx > no:
        turn = PLAYER_O

    # Initialize cache
    cache = [None] * (3 ** 9)

    # Returns whether current turn player can win the game
    # with given state of the board
    def solve(board, turn):
        if judge(board, ~turn):
            return LOSE

        # Fetch cached data
        c_idx = flatten(board)
        ret = cache[c_idx]
        if ret is not None:
            return ret

        # No cached result
        ret = INF
        for y in range(3):
            for x in range(3):
                if board[y][x] is None:
                    board[y][x] = turn
                    # Preference: high) (opposite's) LOSE -> DRAW -> WIN (low
                    ret = min(ret, solve(board, ~turn))
                    board[y][x] = None

        if ret == INF or ret == DRAW:
            ret = cache[c_idx] = DRAW
        else:  # :ret in (WIN, LOSE)
            # :ret is the best action of opposite; invert it for current player
            ret = cache[c_idx] = (-ret)  # WIN == (-LOSE)
        return ret

    result = solve(board, turn)
    if result == DRAW:
        return 'TIE'
    elif result == LOSE:
        turn = ~turn
    return 'o' if turn == PLAYER_O else 'x'


# Main I/O part
def main(rl):
    C = int(rl())
    for _ in range(C):
        board = []
        for _ in range(3):
            board.append(rl())

        result = solution(board)
        print(result)


# Additional codes to simulate I/O
try:
    import IPython
except ImportError as _:
    # Submit env
    import sys
    main(sys.stdin.readline)
else:
    from helpers import runner
    # IPython env
    runner.run(main)

In [4]:
"""
Testing

"""
from helpers import testing

test_cases = [
    {
        'input': [
            ['...', '...', '...']
        ],
        'expected': 'TIE',
        'verbose': True,
    },
    {
        'input': [
            ['xx.', 'oo.', '...']
        ],
        'expected': 'x',
    },
    {
        'input': [
            ['xox', 'oo.', 'x.x']
        ],
        'expected': 'o',
    },
]

testing.run(solution, test_cases)

[0.12219980] solution(['...', '...', '...']) → 'TIE'
Case 1 PASS

         61893 function calls (45726 primitive calls) in 0.129 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    13386    0.055    0.000    0.055    0.000 <ipython-input-3-d7583c5dd190>:26(flatten)
    16168    0.033    0.000    0.033    0.000 <ipython-input-3-d7583c5dd190>:38(judge)
        1    0.000    0.000    0.129    0.129 <ipython-input-3-d7583c5dd190>:56(solution)
  16168/1    0.036    0.000    0.128    0.128 <ipython-input-3-d7583c5dd190>:84(solve)
        1    0.000    0.000    0.129    0.129 <string>:1(<module>)
        1    0.000    0.000    0.129    0.129 {built-in method builtins.exec}
    16167    0.005    0.000    0.005    0.000 {built-in method builtins.min}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}



[0.00064910] solution(['xx.', 'oo.', '...']) → 'x'
Case 2 PASS

[0.00003230] solution([